In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.cuda
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import os
from PIL import Image
from tqdm.notebook import tqdm
from torch.nn.functional import relu
from torch.utils.data import DataLoader, Dataset
from sklearn.manifold import TSNE
from SHG import SHG
from utils import *
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.base import clone
from sklearn_extra.cluster import KMedoids

In [10]:
""" PATHS """
# Input images
TRAIN_IMGS_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/train/image/"

# Validation images
VAL_IMGS_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/validation/image/"

# Path for model
MODEL_DIR = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/models/Mon_Apr_12_18-11-59_2021/epoch_31.pth"

# Path for mean rgb
average_rgb = np.loadtxt("./average_rgb.npy")

# Saving path
SAVING_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data_min_loss/data/"
SAVING_PATH_VAL = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data_min_loss/val/"

In [14]:
""" DATASET AND DATALOADER """

class dataset(Dataset):
    def __init__(self, X_path, average_rgb):
        self.X_path = X_path
        self.X_data = os.listdir(self.X_path)
        self.average_rgb = average_rgb
        self.norm = transforms.Normalize(mean = self.average_rgb, std = [1, 1, 1])

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, i):
        ID = self.X_data[i]
        x = Image.open(self.X_path + ID)

        x = TF.to_tensor(x)

        if (x.shape[0] == 1): # If the image is gray-scale, cast it to rgb
            x = torch.stack((x[0],) * 3)

        x = self.norm(x) # Subtracts mean rgb

        return x, ID

train_data = dataset(TRAIN_IMGS_PATH, average_rgb)
train_dataloader = DataLoader(train_data, batch_size = 1, shuffle = True)

val_data = dataset(VAL_IMGS_PATH, average_rgb)
val_dataloader = DataLoader(val_data, batch_size = 1, shuffle=True)

In [4]:
""" LOADING MODEL """
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SHG(num_hourglasses = 1, use_skip_connections = True).to(device)
model.load_state_dict(torch.load(MODEL_DIR))
model.eval()
print()

In [8]:
""" Saving bottleneck data - train"""
 
with torch.no_grad():
    for x, id_ in tqdm(train_dataloader):
        x = x.to(device, dtype = torch.float)

        _, bottleneck = model(x)

        bottleneck = bottleneck[-1][0].cpu().data.numpy()
        np.save(SAVING_PATH + id_[0][:-4] + ".npy", bottleneck)

In [16]:
""" Saving bottleneck data - validation"""
 
with torch.no_grad():
    for x, id_ in tqdm(val_dataloader):
        x = x.to(device, dtype = torch.float)

        _, bottleneck = model(x)

        bottleneck = bottleneck[-1][0].cpu().data.numpy()
        np.save(SAVING_PATH_VAL + id_[0][:-4] + ".npy", bottleneck)